- To download, evaluate and compare the HF's smolLM-135M model and our trained one

- conda env: 
CONDA_ENV_PATH="/proj/data-eng/granite_docling/conda-envs/fms-fsdp-env" # NEW env: pip list (but install with -e): ibm-fms 1.0.0     /proj/data-eng/granite_docling/foundation-model-stack


CONDA_ENV_PATH="/proj/data-eng/fsdp/train_env" # ab.study env -> use fms which may not support BOTH KVQ and MLP fused! at /proj/data-eng/fsdp/foundation-model-stack/

- ref: 
    + SmolLM2-135M: https://huggingface.co/HuggingFaceTB/SmolLM2-135M


In [2]:
!conda info -e

# conda environments:
#
                         /proj/data-eng/finetune/conda-envs/dpk
                         /proj/data-eng/finetune/conda-envs/env-fms-hf-tuning
                         /proj/data-eng/finetune/conda-envs/instruct-tune-env
                         /proj/data-eng/fms-hf-tuning/train_env
                      *  /proj/data-eng/granite_docling/conda-envs/fms-fsdp-env
                         /proj/data-eng/replaybuffer/07-conda-env/fms-hf-tuning-env
                         /proj/data-eng/replaybuffer/07-conda-env/fms-hf-tuning-env2
                         /proj/data-eng/replaybuffer/07-conda-env/fms-hf-tuning-yashavi-env-clone1
                         /proj/data-eng/replaybuffer/07-conda-env/open-instruct-env
                         /proj/data-eng/replaybuffer/07-conda-env/open-instruct-fastmoe-env
                         /proj/data-eng/replaybuffer/07-conda-env/open-instruct-gg-env
base                     /u/xdang/miniconda3



In [ ]:
'''
Download:
'''

from transformers import AutoTokenizer, AutoModelForCausalLM

checkpoint = "HuggingFaceTB/SmolLM2-135M"
local_dir = "/proj/data-eng/granite_docling/experiments/SmolLM2-135M"

# Download and save the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenizer.save_pretrained(local_dir)

model = AutoModelForCausalLM.from_pretrained(checkpoint)
model.save_pretrained(local_dir)

/proj/data-eng/granite_docling/conda-envs/fms-fsdp-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/proj/data-eng/granite_docling/conda-envs/fms-fsdp-env/lib/python3.12/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/proj/data-eng/granite_docling/conda-envs/fms-fsdp-env/lib/python3.12/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

checkpoint = "HuggingFaceTB/SmolLM2-135M"
local_dir = "/proj/data-eng/granite_docling/experiments/SmolLM2-135M"

# Download and save the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenizer.save_pretrained(local_dir)

model = AutoModelForCausalLM.from_pretrained(checkpoint)
model.save_pretrained(local_dir)

In [16]:
'''
Do inference
'''
import os

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

def run_inference(local_dirs:list, prompt: str) -> str:
    """
    Load local models and tokenizers from a local dir and run inference on a given prompt using CPU.

    Args:
        local_dir (str): Path to the directory where the model and tokenizer are stored.
        prompt (str): The input prompt to generate text from.

    Returns:
        str: The generated text.
    """
    
    print(f"\n== Prompt: {prompt}")
    for local_dir in local_dirs:
        
        tokenizer = AutoTokenizer.from_pretrained(local_dir)
        model = AutoModelForCausalLM.from_pretrained(local_dir)
        
        model_name = os.path.basename(os.path.normpath(local_dir))

        inputs = tokenizer.encode(prompt, return_tensors="pt", truncation=True, max_length=2048)
        try:
            outputs = model.generate(inputs, pad_token_id=tokenizer.eos_token_id)
            # outputs = model.generate(inputs,pad_token_id=tokenizer.eos_token_id,max_new_tokens=200)
        except Exception as gen_err:
                print(f"\t⚠️ Skipped generation for `{model_name}` due to error: {gen_err}")
                continue

        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        print(f"\t -> Response by `{model_name}`: {response}")

evaluated_models = [
    "/proj/data-eng/granite_docling/experiments/SmolLM2-135M",
    "/proj/data-eng/granite_docling/experiments/llama165m_granite4tiktoken-2T-pdbs2-8node/hf_model/step_100000_ckp",
    "/proj/data-eng/granite_docling/experiments/llama165m_granite4tiktoken-2T-pdbs2-8node/hf_model/step_1900000_ckp"
]

run_inference(evaluated_models,"Gravity is")

# run_inference("/proj/data-eng/granite_docling/experiments/SmolLM2-135M","Gravity is")
# run_inference("/proj/data-eng/granite_docling/experiments/llama165m_granite4tiktoken-2T-pdbs2-8node/hf_model/step_100000_ckp","Gravity is")
# run_inference("/proj/data-eng/granite_docling/experiments/llama165m_granite4tiktoken-2T-pdbs2-8node/hf_model/step_1900000_ckp","Gravity is")



== Prompt: Gravity is


/proj/data-eng/granite_docling/conda-envs/fms-fsdp-env/lib/python3.12/site-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


	 -> Response by `SmolLM2-135M`: Gravity is the force that holds the Earth and the Moon together.

The Moon is a
	 -> Response by `step_100000_ckp`: Gravity is the force that holds the Earth together. It is the force that keeps the Earth in orbit
	 -> Response by `step_1900000_ckp`: Gravity is a measure of the resistance of a fluid to flow. The SI unit of pressure is the


In [17]:
test_prompts = [
    "Explain how photosynthesis works in simple terms.",
    "What would happen if gravity on Earth was suddenly cut in half?",
    "Compare DNA and RNA in terms of structure and function.",

    "If a train travels 60 miles per hour for 2.5 hours, how far does it travel?",
    "What is the next number in the sequence: 2, 4, 8, 16, ?",
    "A farmer has 17 sheep. All but 9 run away. How many are left?",

    "Write a Python function that checks if a number is prime.",
    "Explain what a Python decorator is and give an example.",
    "Fix this buggy Python code:\n\nfor i in range(10)\n    print(i)",

    "Summarize the following paragraph in one sentence: 'The climate crisis has accelerated due to industrial emissions and deforestation...'",
    "Paraphrase: 'Artificial intelligence is changing the way businesses operate.'",

    "List three steps to make a peanut butter and jelly sandwich.",
    "Tell me how to reset my Wi-Fi router.",
    "Translate 'Good morning, how are you?' into Spanish.",

    "Write a short poem about the moon.",
    "Continue the story: 'The robot looked at the stars and whispered, \"I remember...\"'",
    "Describe a futuristic city built under the ocean.",

    # Conversation / Chatbot Style
    "Hi, what’s your name?",
    "Tell me a joke!",
    "What’s the weather like today in Paris?"
]


for prompt in test_prompts:
    run_inference(evaluated_models,prompt)


== Prompt: Explain how photosynthesis works in simple terms.
	 -> Response by `SmolLM2-135M`: Explain how photosynthesis works in simple terms.

Photosynthesis is the process by which plants convert light
	 -> Response by `step_100000_ckp`: Explain how photosynthesis works in simple terms. What is the role of the chloroplast in photos
	 -> Response by `step_1900000_ckp`: Explain how photosynthesis works in simple terms. What is the process of photosynthesis? What are

== Prompt: What would happen if gravity on Earth was suddenly cut in half?
	 -> Response by `SmolLM2-135M`: What would happen if gravity on Earth was suddenly cut in half?

The answer is that the
	 -> Response by `step_100000_ckp`: What would happen if gravity on Earth was suddenly cut in half? What would happen if the Earth were
	 -> Response by `step_1900000_ckp`: What would happen if gravity on Earth was suddenly cut in half? Would the Earth still be pulled towards

== Prompt: Compare DNA and RNA in terms of structure

In [1]:
'''
Check spec.tokens
'''

import os

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

evaluated_models = [
    "/proj/data-eng/granite_docling/experiments/SmolLM2-135M",
    "/proj/data-eng/granite_docling/experiments/llama165m_granite4tiktoken-2T-pdbs2-8node/hf_model/step_100000_ckp",
    "/proj/data-eng/granite_docling/experiments/llama165m_granite4tiktoken-2T-pdbs2-8node/hf_model/step_1900000_ckp"
]


local_dir = evaluated_models[0]
model_name = os.path.basename(os.path.normpath(local_dir))

tokenizer = AutoTokenizer.from_pretrained(local_dir)

print(f"Vocab size: {tokenizer.vocab_size:,} {len(tokenizer):,}")
print("Special Tokens:")
print("BOS token:", tokenizer.bos_token, "| ID:", tokenizer.bos_token_id)
print("EOS token:", tokenizer.eos_token, "| ID:", tokenizer.eos_token_id)
print("PAD token:", tokenizer.pad_token, "| ID:", tokenizer.pad_token_id)
print("SEP token:", tokenizer.sep_token, "| ID:", tokenizer.sep_token_id)
print("CLS token:", tokenizer.cls_token, "| ID:", tokenizer.cls_token_id)
print("UNK token:", tokenizer.unk_token, "| ID:", tokenizer.unk_token_id)
print("MASK token:", tokenizer.mask_token, "| ID:", tokenizer.mask_token_id)

# Alternatively, get a dictionary of all special tokens
print("\nAll Special Tokens:", tokenizer.special_tokens_map)

/proj/data-eng/granite_docling/conda-envs/fms-fsdp-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Vocab size: 49,152 49,152
Special Tokens:
BOS token: <|endoftext|> | ID: 0
EOS token: <|endoftext|> | ID: 0
PAD token: None | ID: None
SEP token: None | ID: None
CLS token: None | ID: None
UNK token: <|endoftext|> | ID: 0
MASK token: None | ID: None

All Special Tokens: {'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'additional_special_tokens': ['<|endoftext|>', '<|im_start|>', '<|im_end|>', '<repo_name>', '<reponame>', '<file_sep>', '<filename>', '<gh_stars>', '<issue_start>', '<issue_comment>', '<issue_closed>', '<jupyter_start>', '<jupyter_text>', '<jupyter_code>', '<jupyter_output>', '<jupyter_script>', '<empty_output>']}


In [2]:

local_dir = evaluated_models[2]
model_name = os.path.basename(os.path.normpath(local_dir))

tokenizer = AutoTokenizer.from_pretrained(local_dir)

print(f"Vocab size: {tokenizer.vocab_size:,} {len(tokenizer):,}")
print("Special Tokens:")
print("BOS token:", tokenizer.bos_token, "| ID:", tokenizer.bos_token_id)
print("EOS token:", tokenizer.eos_token, "| ID:", tokenizer.eos_token_id)
print("PAD token:", tokenizer.pad_token, "| ID:", tokenizer.pad_token_id)
print("SEP token:", tokenizer.sep_token, "| ID:", tokenizer.sep_token_id)
print("CLS token:", tokenizer.cls_token, "| ID:", tokenizer.cls_token_id)
print("UNK token:", tokenizer.unk_token, "| ID:", tokenizer.unk_token_id)
print("MASK token:", tokenizer.mask_token, "| ID:", tokenizer.mask_token_id)

# Alternatively, get a dictionary of all special tokens
print("\nAll Special Tokens:", tokenizer.special_tokens_map)

Vocab size: 100,352 100,352
Special Tokens:
BOS token: <|end_of_text|> | ID: 100257
EOS token: <|end_of_text|> | ID: 100257
PAD token: <|pad|> | ID: 100256
SEP token: None | ID: None
CLS token: None | ID: None
UNK token: <|unk|> | ID: 100269
MASK token: None | ID: None

All Special Tokens: {'bos_token': '<|end_of_text|>', 'eos_token': '<|end_of_text|>', 'unk_token': '<|unk|>', 'pad_token': '<|pad|>'}


In [3]:
'''
Original granite4 tiktoken:
'''

from transformers import PreTrainedTokenizerFast

# Load the tokenizer
tokenizer_path = "/proj/data-eng/tokenizers/granite4tiktoken"
tokenizer = PreTrainedTokenizerFast.from_pretrained(tokenizer_path)

# Print tokenizer information
print("Tokenizer Info")
print("==============")
print(f"Tokenizer class: {tokenizer.__class__.__name__}")
print(f"Vocab size: {tokenizer.vocab_size}")
print(f"Model max length: {tokenizer.model_max_length}")
print(f"Padding token: {tokenizer.pad_token} (ID: {tokenizer.pad_token_id})")
print(f"BOS token: {tokenizer.bos_token} (ID: {tokenizer.bos_token_id})")
print(f"EOS token: {tokenizer.eos_token} (ID: {tokenizer.eos_token_id})")
print(f"UNK token: {tokenizer.unk_token} (ID: {tokenizer.unk_token_id})")
print("\nSpecial tokens:")
for name, tok in tokenizer.special_tokens_map.items():
    print(f"  {name}: {tok}")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2TokenizerFast'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


Tokenizer Info
Tokenizer class: PreTrainedTokenizerFast
Vocab size: 100352
Model max length: 1000000000000000019884624838656
Padding token: <|pad|> (ID: 100256)
BOS token: <|end_of_text|> (ID: 100257)
EOS token: <|end_of_text|> (ID: 100257)
UNK token: <|unk|> (ID: 100269)

Special tokens:
  bos_token: <|end_of_text|>
  eos_token: <|end_of_text|>
  unk_token: <|unk|>
  pad_token: <|pad|>
